# Spark SQL

Useful functionality where you can use SQL expressions on your tables.

```python
df1.createOrReplaceTempView("some_dataframe")
sqlDF = spark.sql("SELECT * FROM some_dataframe")

```

In [2]:
import sys
sys.path.append('/home/iceberg/notebooks/PyCon_LT_Workshop')

from helpers.utils import get_spark_session, get_yellow_taxi_data, get_dim_data
from pyspark.sql import functions as f
spark = get_spark_session("joins")

yellow_taxi_data = get_yellow_taxi_data(spark=spark)

dim_taxi_zones, dim_rates, dim_payments, dim_vendor = get_dim_data(spark)



24/02/17 08:59:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
yellow_taxi_data.createOrReplaceTempView("yellow_taxi")
dim_vendor.createOrReplaceTempView("dim_vendor")

In [4]:
spark.sql("SELECT count(1) FROM yellow_taxi").head()

Row(count(1)=5443362)

In [10]:
joined = spark.sql("select VendorID, vendor_name from yellow_taxi join dim_vendor on yellow_taxi.VendorID = dim_vendor.vendor_id")


In [11]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf, col


@udf(returnType=IntegerType())
def vendor_name_length(name):
    return len(name) if name else 0

In [19]:
spark.udf.register("vendor_name_length", vendor_name_length)
joined.createOrReplaceTempView("joined")

24/02/17 09:03:18 WARN SimpleFunctionRegistry: The function vendor_name_length replaced a previously registered function.


In [21]:
spark.sql("SELECT VendorID,vendor_name, vendor_name_length(vendor_name) FROM joined").show()

+--------+--------------------+-------------------------------+
|VendorID|         vendor_name|vendor_name_length(vendor_name)|
+--------+--------------------+-------------------------------+
|       1|Creative Mobile T...|                             28|
|       1|Creative Mobile T...|                             28|
|       2|       VeriFone Inc.|                             13|
|       2|       VeriFone Inc.|                             13|
|       2|       VeriFone Inc.|                             13|
|       1|Creative Mobile T...|                             28|
|       2|       VeriFone Inc.|                             13|
|       2|       VeriFone Inc.|                             13|
|       2|       VeriFone Inc.|                             13|
|       2|       VeriFone Inc.|                             13|
|       2|       VeriFone Inc.|                             13|
|       2|       VeriFone Inc.|                             13|
|       2|       VeriFone Inc.|         